我们展示一个35比特的线路，用cirq模拟将需要256GB内存，导致cirq报错无法模拟。而使用tensorq可以完成模拟。

首先，我们用cirq模拟10比特的类似线路，验证代码的正确性。

In [1]:
import cirq

# Set n qubit.
n = 10
qubits = [cirq.GridQubit(0, i) for i in range(n)]

# Create a circuit
# circuit = cirq.Circuit(cirq.Moment([cirq.H(qubits[0])]))
circuit = cirq.Circuit(
    moments = [
        cirq.Moment([cirq.H(qubits[0])]),
        (cirq.Moment([cirq.CNOT(qubits[i], qubits[i+1])]) for i in range(n-1))
    ]
)
print("Circuit:")
print(circuit)

# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
print("Results:")
print(result)

Circuit:
(0, 0): ───H───@───────────────────────────────────
               │
(0, 1): ───────X───@───────────────────────────────
                   │
(0, 2): ───────────X───@───────────────────────────
                       │
(0, 3): ───────────────X───@───────────────────────
                           │
(0, 4): ───────────────────X───@───────────────────
                               │
(0, 5): ───────────────────────X───@───────────────
                                   │
(0, 6): ───────────────────────────X───@───────────
                                       │
(0, 7): ───────────────────────────────X───@───────
                                           │
(0, 8): ───────────────────────────────────X───@───
                                               │
(0, 9): ───────────────────────────────────────X───
Results:
measurements: (no measurements)
output vector: 0.707|0000000000⟩ + 0.707|1111111111⟩


/tmp/ipykernel_15603/446852532.py:9: DeprecationWarning: The moments keyword parameter of cirq.Circuit was used but is deprecated.
It will be removed in cirq v0.8.
Pass circuit contents positionally (without a keyword).

  circuit = cirq.Circuit(


接下来我们把n设置为35，表示模拟35比特的线路

In [2]:
import cirq

# Set n qubit.
n = 35
qubits = [cirq.GridQubit(0, i) for i in range(n)]

# Create a circuit
# circuit = cirq.Circuit(cirq.Moment([cirq.H(qubits[0])]))
circuit = cirq.Circuit(
    moments = [
        cirq.Moment([cirq.H(qubits[0])]),
        (cirq.Moment([cirq.CNOT(qubits[i], qubits[i+1])]) for i in range(n-1))
    ]
)
print("Circuit:")
print(circuit)

# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
print("Results:")
print(result)

/tmp/ipykernel_15603/2928650337.py:9: DeprecationWarning: The moments keyword parameter of cirq.Circuit was used but is deprecated.
It will be removed in cirq v0.8.
Pass circuit contents positionally (without a keyword).

  circuit = cirq.Circuit(


Circuit:
(0, 0): ────H───@───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                │
(0, 1): ────────X───@───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                    │
(0, 2): ────────────X───@───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                        │
(0, 3): ────────────────X───@───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                            │
(0, 4): ────────────────────X───@───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                │
(0, 5): ────────────────────────X───@───────────────────────────────────────────────────────────

MemoryError: Unable to allocate 256. GiB for an array with shape (34359738368,) and data type complex64

报错！需要256GB内存，无法模拟！

接下来，我们用tensorq模拟同样的线路

首先在`circuit.py`文件中构建量子线路，然后用tensorq中的QuantumCircuit将量子子线路转换成张量网络（注意修改`circuit.py`文件后，要Restart，tensorq才能读取到新线路）

In [3]:
from tensorq import QuantumCircuit
n = 35
qc_n35 = QuantumCircuit(n = n,twoqubit_simplify = False,fname = 'circuit')

其中，n = 35表示35个比特；twoqubit_simplify = False表示不进行两比特门优化，因为该线路中两比特门不必优化；fname = 'circuit'表示线路在文件`circuit.py`中。

用tensorq中的search_order搜索缩并方案

In [4]:
from tensorq import search_order
contract_scheme = search_order(n = n,device = 'cuda',qc = qc_n35, sc_target = 24, bitstrings_txt = 'two_bitstrings_n35.txt', max_bitstrings=2)

time complexity (tc,log10), space complexity (sc,log2), memory complexity (mc) =  (3.1535099893008374, 4.0, 3.338456493604605)


其中，n = n表示n个比特；device = 'cuda' 表示用GPU进行计算，用CPU计算使用device = 'cpu'；qc = qc_n35 输入之前定义的量子线路。

sc_target = 24表示限制最大张量小于24GB内存，这是tensorq中很核心的一个参数，可以根据自己的设备内存进行设置。同样的缩并任务，内存越大时间复杂度越低，内存越小将会对张量做更多的分解导致更多的浮点运算，这是用时间换空间的办法解决量子模拟中的指数墙困难。

bitstrings_txt = 'two_bitstrings_n35.txt' 表示将我们需要计算振幅的比特串写在`two_bitstrings_n35.txt`文件里面，因为只有'00000000000000000000000000000000000', '11111111111111111111111111111111111' 两个比特串的振幅非0，因此这里只计算两个比特串的振幅。

max_bitstrings=2，表示最多计算2两个比特串的振幅。

search_order会打印该缩并方案的复杂度，时间复杂度为浮点数运算次数，以10为底的对数表示；空间复杂度为最大张量的元素个数（与sc_target单位并不相同），以2为底的对数表示；空间复杂度为最大张量元素个数乘以单个数据的存储大小（与数据类型有关）。


最后，用tensorq的contraction_single_task对张量进行缩并。

In [5]:
from tensorq import contraction_single_task
tensors, scheme, slicing_indices, bitstrings = contract_scheme
result = contraction_single_task(
    tensors,
    scheme,
    slicing_indices
)
print('fianl amplitue:', result)

fianl amplitue: tensor([0.7071+0.j, 0.7071+0.j])


contract_scheme中包含了缩并张量tensors，缩并步骤scheme，切片指标slicing_indices（与张量分解有关），计算振幅的比特串bitstrings，将这些变量输入到contraction_single_task，就会返回张量网络的缩并结果，就是全部末态比特串的振幅。


In [6]:
slicing_indices

{}

在这个例子中，由于目标比特串比较少，仅通过调整张量网络的缩并顺序就可以避免出现较大的张量，因此search_order中只优化了缩并顺序，并没有采用切片策略来减小张量维度